In [ ]:
inference_prompt_style = """Below is a fixed instruction that guides the assistant to write a Korean patent specification in full compliance with Korean Patent Law. 
The assistant must use the provided invention details to generate a structured patent section in Korean.
Before answering, think step-by-step to ensure a logically valid and legally compliant response.
**Respond in Korean.**

### Instruction:
당신은 대한민국 특허법에 따라 명세서를 작성하는 특허 어시스턴트입니다. 아래의 구성요소별로 정확하고 구조화된 문서를 생성해야 합니다. 
모든 항목은 특허법 제42조 및 시행규칙 제21조, 개정된 모범 명세서 작성법(2007.07.01. 이후 적용)을 철저히 준수해야 합니다.

1. [발명의 명칭]
- 발명의 내용을 간명하게 표현하는 명칭을 작성합니다.
- 영문명을 {{}} 안에 함께 기재합니다.

2. [기술분야]
- 본 발명이 속하는 기술분야를 간결하게 설명합니다.

3. [배경기술]
- 종래 기술을 서술하고, 가능하면 문헌 인용을 포함합니다.

4. [해결하려는 과제]
- 기존 기술의 문제점과 해결하고자 하는 과제를 기술합니다.

5. [과제의 해결 수단]
- 본 발명이 과제를 어떻게 해결하는지를 구체적으로 기술하고, 청구항과 연결합니다.

6. [발명의 효과]
- 본 발명이 제공하는 구체적이고 비교 가능한 기술적 효과를 기술합니다.

7. [발명을 실시하기 위한 구체적인 내용]
- 실시예, 수단, 수치 등을 포함하여, 평균 기술자가 재현 가능하도록 작성합니다.

8. [도면의 간단한 설명]
- 도면이 있을 경우, 각 도면이 무엇을 나타내는지 기술합니다.

9. [특허청구범위]
- 독립항 및 종속항을 번호와 함께 명확히 구분하여 작성합니다.

※ 모든 기재는 특허법 제42조 제2~5항, 시행규칙 제21조 및 모범명세서 가이드를 기반으로 작성해야 합니다.

### Content:
없음

### Input:
{input}

### Response:
{output}
"""


In [ ]:
import json
from datasets import Dataset

# ✅ 데이터 로딩
def load_jsonl_dataset(path):
    with open(path, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return Dataset.from_list(data)

dataset = load_jsonl_dataset("testcase_data.jsonl")

# ✅ 하나의 샘플 선택
input_text = dataset[10]['input']
prompt = inference_prompt_style.format(input=input_text) + tokenizer.eos_token

# ✅ Tokenize
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# ✅ Generate
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1024 * 8,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True
)

# ✅ Decode
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-8B-Base", device_map="auto")
model = PeftModel.from_pretrained(base_model, "best_model")

tokenizer = AutoTokenizer.from_pretrained("best_model")


In [2]:
# ✅ 필요한 패키지 임포트
import json
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# ✅ 1. 모델 및 토크나이저 로드
base_model_name = "Qwen/Qwen3-8B"        # 원본 base 모델
adapter_model_path = "best_model"            # LoRA 학습된 adapter 경로

# 토크나이저
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # padding 설정

# base 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# adapter 로드
model = PeftModel.from_pretrained(base_model, adapter_model_path)
model.eval()

# ✅ 2. 프롬프트 템플릿
inference_prompt_style = """Below is a fixed instruction that guides the assistant to write a Korean patent specification in full compliance with Korean Patent Law. 
The assistant must use the provided invention details to generate a structured patent section in Korean.
Before answering, think step-by-step to ensure a logically valid and legally compliant response.
**Respond in Korean.**

### Instruction:
당신은 대한민국 특허법에 따라 명세서를 작성하는 특허 어시스턴트입니다. 아래의 구성요소별로 정확하고 구조화된 문서를 생성해야 합니다. 
모든 항목은 특허법 제42조 및 시행규칙 제21조, 개정된 모범 명세서 작성법(2007.07.01. 이후 적용)을 철저히 준수해야 합니다.

1. [발명의 명칭]
- 발명의 내용을 간명하게 표현하는 명칭을 작성합니다.
- 영문명을 {{}} 안에 함께 기재합니다.

2. [기술분야]
- 본 발명이 속하는 기술분야를 간결하게 설명합니다.

3. [배경기술]
- 종래 기술을 서술하고, 가능하면 문헌 인용을 포함합니다.

4. [해결하려는 과제]
- 기존 기술의 문제점과 해결하고자 하는 과제를 기술합니다.

5. [과제의 해결 수단]
- 본 발명이 과제를 어떻게 해결하는지를 구체적으로 기술하고, 청구항과 연결합니다.

6. [발명의 효과]
- 본 발명이 제공하는 구체적이고 비교 가능한 기술적 효과를 기술합니다.

7. [발명을 실시하기 위한 구체적인 내용]
- 실시예, 수단, 수치 등을 포함하여, 평균 기술자가 재현 가능하도록 작성합니다.

8. [도면의 간단한 설명]
- 도면이 있을 경우, 각 도면이 무엇을 나타내는지 기술합니다.

9. [특허청구범위]
- 독립항 및 종속항을 번호와 함께 명확히 구분하여 작성합니다.

※ 모든 기재는 특허법 제42조 제2~5항, 시행규칙 제21조 및 모범명세서 가이드를 기반으로 작성해야 합니다.

### Content:
없음

### Input:
{input}

### Response:
"""

# ✅ 3. 테스트 데이터 로딩
def load_jsonl_dataset(path):
    with open(path, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return Dataset.from_list(data)

dataset = load_jsonl_dataset("testcase_data.jsonl")  # ✅ input 필드를 포함해야 함

# ✅ 4. 하나의 샘플을 선택하여 프롬프트 생성
sample_idx = 0
input_text = dataset[sample_idx]["input"]
prompt = inference_prompt_style.format(input=input_text) + tokenizer.eos_token

# ✅ 5. 토크나이징 및 인퍼런스
inputs = tokenizer([prompt], return_tensors="pt", padding=True).to("cuda")
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=2048,
        do_sample=True, 
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True,
    )

# ✅ 6. 결과 디코딩 및 출력
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print("\n\n📌 최종 생성 결과:")
print("-" * 80)
print(decoded.split("### Response:")[-1].strip())


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(




📌 최종 생성 결과:
--------------------------------------------------------------------------------
**1. [발명의 명칭]**  
인공지능 모델을 이용한 육류 등급 판단 방법 및 시스템  
{AI-based Meat Grade Evaluation Method and System Using Machine Learning}

**2. [기술분야]**  
본 발명은 인공지능 기술을 활용하여 육류의 품질을 객관적으로 평가하는 방법 및 시스템에 관한 것으로, 특히 육류 이미지 분석을 통해 마블링, 육색, 조직감 등의 특성을 자동으로 평가하여 등급을 판단하는 기술에 해당합니다. 이 기술은 육류 산업에서의 품질 관리 및 소비자 선택의 편의성을 높이는 데 기여합니다.

**3. [배경기술]**  
기존의 육류 등급 평가 방식은 주로 전문가의 시각에 의존하여 이루어졌으며, 이로 인해 평가의 주관성과 일관성이 문제가 되었습니다. 최근에는 컴퓨터 비전 기술이 발전함에 따라, 육류 이미지를 분석하여 자동화된 등급 판단 시스템이 연구되고 있으나, 여전히 정확도와 신뢰성의 한계가 존재합니다. 예를 들어, 기존의 연구에서는 특정 육류 종류에 한정된 모델이 주로 사용되어, 다양한 육류에 대한 일반화된 솔루션은 부족했습니다.

**4. [해결하려는 과제]**  
육류 소비의 증가로 소비자들은 품질에 대한 요구가 높아지고 있으며, 기존의 주관적인 평가 방식은 소비자에게 혼란을 초래하고 육류 구매 결정에 부정적인 영향을 미칩니다. 본 발명은 인공지능 모델을 통해 객관적이고 일관된 육류 등급 평가를 제공함으로써 이러한 문제를 해결하고, 소비자에게 신뢰할 수 있는 정보를 제공하여 선택의 정확성을 높이는 것을 목표로 합니다.

**5. [과제의 해결 수단]**  
본 발명은 인공지능 모델을 기반으로 육류 이미지를 분석하여 등급을 자동으로 판단하는 시스템을 제공합니다. 학습 데이터 세트를 생성하고, 이를 기반으로 모델을 훈련시켜 